# Desarollo Problemas Grafos

## Problema 1

La teoría de los seis grados de separación dice que una persona podría conocer a cualquier otra
persona del mundo siguiendo una cadena de personas que se conocen entre si de tamaño
máximo 6. Suponiendo que una persona conoce a otra si y solo si tienen una relación de amistad
en la red social Facebook, diseñar un algoritmo que reciba la base de datos de relaciones de
amistad de esta red social y determine si la teoria de los seis grados de separación se cumple.


### Entradas y Salidas

| E/S | Nombre | Tipo | Descripción |
|-----|--------|------|-------------|
| E   |   G    |  List of **tuples** of **string** | Parejas de personas que son amigas en Facebook |
| E   |   N    |  List of **string**  | Lista de personas involucradas en la red social |
| S   |   b    | **boolean** | Representa si se cumple la regla de los seis grados de separación |

### Estructura de grafo a utilizar

En este problema tendremos un grafo que tendrá los siguientes elementos:

* Nodos o Vertices: Representan las diferentes personas que están en la red social facebook.
* Arcos o Aristas: Representan si existe una relación de amistad entre dos personas en Facebook.
* Dirigido: El grafo a representar es no dirigido, ya que para tener una relación de amistad, esta se debio generar por ambas partes (En otras redes sociales donde se tiene el concepto de seguidor se trataría de forma diferente, pero al estar en el contexto de facebook, se puede tratar como un grafo no dirigido)

### Diseño de algoritmo a utilizar

Para este problema, debemos identificar que el grafo es conexo y que para todo nodo, su distancia minima es mayor o igual a 6. Dado que el grafo es no dirigido y no tiene costos, se puede hacer esta tarea usando BFS o DFS.

Se tomo la decisión de implementar BFS para solucionar este problema

In [147]:
from collections import deque

class Graph:
    def __init__(self, edges, nodes):
        self.nodes = nodes
        self.edges = set(edges)
        self.Adj = {n:set() for n in nodes}
        for source, destiny in edges:
            self.Adj[source].add(destiny)
            self.Adj[destiny].add(source)

def BFSCheck(g,u):
    states = {}
    distance = {}
    for n in g.nodes:
        states[n]="No Visitado"
        distance[n] = float("inf")
    
    q = deque()
    q.append(u)
    states[u] = "Visitado"
    distance[u] = 0
    visited = 1
    while len(q)!=0:
        u = q.popleft()
        for v in g.Adj[u]:
            if states[v]=="No Visitado":
                q.append(v)
                visited+=1
                distance[v] = distance[u]+1
                states[v]="Visitado"
                if distance[v]>6:
                    return False
    return visited == len(g.nodes) 

def sixDegreeCheck(g):
    for node in g.nodes:
        b = BFSCheck(g,node)
        if not b: return False
    return True

In [148]:
people = [f"Persona{i}" for i in range(1,9)]
friendship = [
    ("Persona1","Persona2"),
    ("Persona2","Persona3"),
    ("Persona3","Persona4"),
    ("Persona4","Persona5"),
    ("Persona5","Persona6"),
    ("Persona6","Persona7"),
    ("Persona7","Persona8")
]
g = Graph(friendship, people)
b = sixDegreeCheck(g)

if b: print("En la red se cumple la teoría de los seis grados de separación")
else: print("En la red no se cumple la teoría de los seis grados de separación")

En la red no se cumple la teoría de los seis grados de separación


In [149]:
people = [f"Persona{i}" for i in range(1,8)]
friendship = [
    ("Persona1","Persona2"),
    ("Persona1","Persona3"),
    ("Persona1","Persona4"),
    ("Persona1","Persona5"),
    ("Persona3","Persona6"),
    ("Persona6","Persona7")
]
g = Graph(friendship, people)
b = sixDegreeCheck(g)

if b: print("En la red si se cumple la teoría de los seis grados de separación")
else: print("En la red no se cumple la teoría de los seis grados de separación")

En la red si se cumple la teoría de los seis grados de separación


## Problema 2

La Superintendencia Bancaria tiene un registro de préstamos que cada entidad bancaria hace a
otra en el país. Con esta información, la Superintendencia está interesada en detectar si hay
autopréstamos en el sistema. Además de casos de prestamos directos de una entidad a si misma,
una autoprestamo también puede ser un esquema en el que una entidad se presta plata a si
misma a traves de una cadena de prestamos que inicia y termina en la misma entidad. Dada la
relación p de prestamos entre entidades, determinar si existe algún autopréstamo.


### Entradas y Salidas

| E/S | Nombre | Tipo | Descripción |
|-----|--------|------|-------------|
| E   |   G    |  List of **tuples**  | Relaciones de préstamos entre las entidades bancarias |
| E   |   N    |  List of **string**  | Lista de entidades bancarias |
| S   |   b    | **boolean** | Representa si existe algun autoprestamo de alguna entidad |

### Estructura de grafo a utilizar

En este problema tendremos un grafo que tendrá los siguientes elementos:

* Nodos o Vertices: Representan las diferentes entidades bancarias que entran a ser estudiadas en el problema.
* Arcos o Aristas: Representan si existe un prestamo desde una entidad bancaria $A$ a una entidad bancaria $B$, su costo representa el valor del prestamo (Costo 0 representa que no hay prestamo).
* Dirigido: El grafo a representar es dirigido ya que el sentido de un prestamo es importante (No es lo mismo que $A$ le preste a $B$ que $B$ le preste a $A$)

### Diseño del Algoritmo a utilizar

El algoritmo que se diseñara determinara si existen algun bucle en el grafo en el que hubo prestamos. Un bucle representa que hubo cierta cantidad de dinero que se presto desde $A$ y que volvio a $A$ pasando por 0 o mas nodos distintos.

Para nuestro propósito, se utilizará el algoritmo de DFS para identificar los bucles. El enfoque se relizara bajo el enfoque de identificar caminos, e individualmente determinar si tienen un ciclo o no. 

In [150]:
class Graph:
    def __init__(self, list, nodes):
        self.nodes = nodes
        self.Adj = {n:[] for n in nodes}
        for source, destiny, _ in list:
            if source not in self.Adj: 
                self.Adj[source] = []
            if destiny not in self.Adj: 
                self.Adj[destiny] = []

            self.Adj[source].append(destiny)

        

def DFSCheckLoop(g):
    color = {}
    for u in g.nodes:
        color[u] = 'WHITE'

    for u in g.nodes:
        if color[u]=='WHITE':
            if DFSCheckPathLoop(g,u,color): return True
    return False

def DFSCheckPathLoop(g,u,color):
    color[u] = 'GRAY'
    for v in g.Adj[u]:
        if color[v] == 'GRAY': return True
        elif color[v] == 'WHITE':
            if DFSCheckPathLoop(g,v,color): return True
    color[u] = 'BLACK'
    return False

In [151]:
p = [
    ('Banco1' , 'Banco2', 130),
    ('Banco1' , 'Banco3', 130),
    ('Banco3' , 'Banco4', 130),
    ('Banco2' , 'Banco4', 130)
]

bancos = [f'Banco{i}' for i in range(1,5)]

graph = Graph(p, bancos)
b = DFSCheckLoop(graph)
if b: print('Se identifico un autoprestamo entre los bancos')
else: print('No se identifico un autoprestamo entre los bancos')

No se identifico un autoprestamo entre los bancos


In [152]:
p = [
    ('Banco1' , 'Banco2', 130),
    ('Banco1' , 'Banco3', 130),
    ('Banco3' , 'Banco4', 130),
    ('Banco2' , 'Banco4', 130),
    ('Banco4' , 'Banco1', 130)
]

bancos = [f'Banco{i}' for i in range(1,5)]

graph = Graph(p, bancos)
b = DFSCheckLoop(graph)
if b: print('Se identifico un autoprestamo entre los bancos')
else: print('No se identifico un autoprestamo entre los bancos')

Se identifico un autoprestamo entre los bancos


## Problema 3

Juan quiere invitar a sus amigos a conocer su nuevo apartamento. Sin embargo tiene la
dificultad de que sus amigos son algo conflictivos y entonces sabe que varias parejas de amigos
se han peleado entre ellos. Debido a esto, tomó la decisión de organizar dos reuniones. Diseñe un
algoritmo que determine si es posible distribuir a los amigos de Juan en dos grupos de tal manera
que dentro de cada grupo no haya parejas de personas que se hayan peleado entre ellas.

### Entradas y Salidas

| E/S | Nombre | Tipo | Descripción |
|-----|--------|------|-------------|
| E   |   G    |  List of **tuples** of **string**   | Lista de personas que están peleadas |
| E   |   N    |  List of **string**   | Lista de amigos de Juan |
| S   |   b    | **boolean** | Representa si se pueden armar dos grupos de personas donde no hayan individuos peleados |
| S   |   G    | Set of **string** | Conjunto de personas que pueden ir en un día (Si no es posible, se devuelve como None o nulo) |

### Estructura del grafo a utilizar

En este problema tendremos un grafo que tendrá los siguientes elementos:

* Nodos o Vertices: Representan las diferentes personas que son amigos de Juan.
* Arcos o Aristas: Representan si dos personas estan peleadas o no.
* Dirigido: Se hace la suposición de que si dos personas están peleadas, ambas tienen conocimiento de que no pueden estar con la otra persona; por consiguiente, es una relación bidireccional y permite denotar que dos personas están peleadas.

### Diseño del Algoritmo a utilizar



El querer saber saber si es posible armar dos grupos de personas que no esten peleadas, es equivalente a saber si un grafo es bipartito, esto se debe a que si una arista representa que dos personas están peleadas, el ideal es que todos los arcos pasen de un conjunto a otro, suponiendo que se busca armar dos conjuntos de personas disjuntos.

Dado esto, se implementará el algoritmo BFS para determinar si el grafo que representa las peleas es bipartito o no.

In [153]:
from collections import deque

class Graph:
    def __init__(self, edges, nodes):
        self.nodes = nodes
        self.Adj = {n:[] for n in nodes}
        for source, destiny in edges:
            self.Adj[source].append(destiny)
            self.Adj[destiny].append(source)
        

def BFSCheckBipartite(g):
    state = {u:"Not Visited" for u in g.nodes} #Assigned to G1 or G2 or not visited
    group_1 = set()

    for u in g.nodes:

        if state[u]!="Not Visited": continue

        q = deque()
        state[u] = "G1"
        q.append(u)

        while len(q)!=0:
            n = q.popleft()
            for v in g.Adj[n]:
                if state[v]=="Not Visited":
                    q.append(v)
                    #Assign different group as adjacent
                    state[v] = "G2" if state[n]=="G1" else "G1"
                #If both adjacent nodes are in the same group, graph is not bipartite 
                elif state[v]==state[n]:
                    return False, None
            if state[n]=="G1": group_1.add(n)
            
    return True, group_1    

In [154]:
friendsFights = [
    ("Juanes" , "Lucas"),
    ("Lucia" , "Juanes"),
    ("Adolfo" , "Juanes"),
    ("David" , "Lucas"),
    ("Adolfo","Jaime")
]
friends = ["Juanes","Lucas","Lucia","David","Adolfo","Jaime"]
g = Graph(friendsFights,friends)
b, group = BFSCheckBipartite(g)

if b: print(f'Juan puede llevar a sus amigos en dos días. El primer día puede llevar a {group}')
else: print(f'Juan no puede llevar a sus amigos en dos días')

Juan puede llevar a sus amigos en dos días. El primer día puede llevar a {'David', 'Juanes', 'Jaime'}


In [155]:
friendsFights = [
    ("Juanes" , "Lucas"),
    ("Lucia" , "Juanes"),
    ("Adolfo" , "Juanes"),
    ("David" , "Lucas"),
    ("Adolfo","Lucia")
]
friends = ["Juanes","Lucas","Lucia","David","Adolfo"]
g = Graph(friendsFights, friends)
b, group = BFSCheckBipartite(g)

if b: print(f'Juan puede llevar a sus amigos en dos días. El primer día puede llevar a {group}')
else: print(f'Juan no puede llevar a sus amigos en dos días')

Juan no puede llevar a sus amigos en dos días


## Problema 4

En una red hay N computadores de los cuales se sabe que se puede enviar un mensaje de
cualquier computador a otro gracias a un conjunto de M conexiones punto a punto. Aunque no hay
conexiones directas entre todos los computadores, un computador puede enviar mensajes a otro
siguiendo una cadena de conexiones. Se quiere saber si existe alguna conexión que sea
indispensable, es decir, si existe alguna conexión entre dos computadores que, en caso de caerse,
impida que se puedan enviar mensajes entre algun par de computadores.

### Entradas y Salidas

| E/S | Nombre | Tipo | Descripción |
|-----|--------|------|-------------|
| E   |   G    |  List of **tuples** of **string**   | Lista de conexiones directas entre equipos |
| E   |   N    |  List of **string**   | Lista de identificadores de computadores |
| S   |   b    | **boolean** | Representa si existe alguna conexión que sea indispensable o no |
| S   |   c    | List of **tuples** of **string** | Lista de conexiones directas indispensables |

### Estructura de grafo a utilizar

En este problema tendremos un grafo que tendrá los siguientes elementos:

* Nodos o Vertices: Representan los diferentes computadores que componen la red.
* Arcos o Aristas: Representan si dos computadores estan conectados de forma directa.
* Dirigido: Por el concepto de red, si un computador esta conectado con otro, ambos se pueden enviar información de manera bidireccional. Por consiguiente, no existe una dirección de la conexión; lo que deriva en que el grafo es no dirigido.

### Diseño del Algoritmo a utilizar

El ver si existe un eje que es indispensable es equivalente a verificar si el grafo mantiene la propiedad de ser conexo si yo retiro el eje del mismo. Por consiguiente, el enfoque a tratar es pata quede eje del grafo, removerlo y verificar que el grafo mantenga su propiedad de ser conexo. En el momento que se identifique un nodo que no cumpla esta propiedad, se catalogará dicho eje como un nodo indispensable.

Para ello se utilizará eel algoritmo BFS para determinar si el grafoo es conexo; y se realizará el ejercicio descrito para cada eje del grafo

In [156]:
from collections import deque

class Graph:
    def __init__(self, edges, nodes):
        self.nodes = nodes
        self.edges = set(edges)
        self.Adj = {n:set() for n in nodes}
        for source, destiny in edges:
            self.Adj[source].add(destiny)
            self.Adj[destiny].add(source)
    
    def removeEdge(self,edge):
        if edge in self.edges:
            self.edges.remove(edge)
            source, destiny = edge
            self.Adj[source].remove(destiny)
            self.Adj[destiny].remove(source)

    def addEdge(self,edge):
        if edge not in self.edges:
            self.edges.add(edge)
            source, destiny = edge
            self.Adj[source].add(destiny)
            self.Adj[destiny].add(source)

def hasIndispensable(g):
    indispensable = set()
    for edge in g.edges:
        g.removeEdge(edge)
        b = isConnectedBFS(g)
        if not b: indispensable.add(edge)
        g.addEdge(edge)
    return len(indispensable)!=0 , indispensable 

def isConnectedBFS(g):
    state = {u:"No Visitado" for u in g.nodes}
    u = g.nodes[0]
    state[u] = "Visitado"
    total = 1
    q = deque()
    q.append(u)

    while len(q)!=0:
        u = q.popleft()
        for v in g.Adj[u]:
            if state[v]=="No Visitado":
                q.append(v)
                total+=1
                state[v] = "Visitado"
                
    return total==len(g.nodes)

In [157]:
computers = [f"Computer{i}" for i in range(1,7)]
connections = [
    ("Computer1","Computer2"),
    ("Computer2","Computer3"),
    ("Computer3","Computer4"),
    ("Computer4","Computer5"),
    ("Computer5","Computer6"),
    ("Computer6","Computer1")
]

g = Graph(connections, computers)
b,indispensable = hasIndispensable(g)

if b: print(f'Se identificaron conexiones indispensables. Estas son:\n {indispensable}')
else: print(f'No se identificaron conexiones indispensables')


No se identificaron conexiones indispensables


In [158]:
computers = [f"Computer{i}" for i in range(1,8)]
connections = [
    ("Computer1","Computer2"),
    ("Computer2","Computer3"),
    ("Computer3","Computer4"),
    ("Computer4","Computer5"),
    ("Computer5","Computer6"),
    ("Computer6","Computer1"),
    ("Computer1","Computer7")
]

g = Graph(connections, computers)
b,indispensable = hasIndispensable(g)

if b: print(f'Se identificaron conexiones indispensables. Estas son:\n {indispensable}')
else: print(f'No se identificaron conexiones indispensables')


Se identificaron conexiones indispensables. Estas son:
 {('Computer1', 'Computer7')}
